## $\underline{\textbf{SQL Notes}}$

<h1 align="center">Queries</h1> 
statement declaring what data to look for, where in database, and optionally how to transform it before it's returned.

$\underline{\textbf{simple select query for specific columns}}:$  
``` sql
SELECT column_name, another_column, …
FROM mytable;
```
#can use asteric (*) to indicate all.

$\underline{\textbf{select query w/ constraint}}:$
``` sql
SELECT column_name, another_column, …
FROM mytable
WHERE condition
    AND/OR another_condition
    AND/OR ..;
```
might find these operators useful: 
| Operator            | Condition                                            | SQL Example                   |
|---------------------|------------------------------------------------------|-------------------------------|
| =, !=, < <=, >, >=  | Standard numerical operators                         | col_name != 4                 |
| BETWEEN … AND …     | Number is within range of two values (inclusive)     | col_name BETWEEN 1.5 AND 10.5 |
| NOT BETWEEN … AND … | Number is not within range of two values (inclusive) | col_name NOT BETWEEN 1 AND 10 |
| IN (…)              | Number exists in a list                              | col_name IN (2, 4, 6)         |
| NOT IN (…)          | Number does not exist in a list                      | col_name NOT IN (1, 3, 5)     |

expanded operator table:
| Operator            | Condition                                            									| SQL Example |
|---------------------|-----------------------------------------------------------------------------------------|-------------|
| = 			      |Case sensitive exact string comparison (notice the single equals) 				|col_name = "abc"     |
| != or <> 		    |Case sensitive exact string inequality comparison     								|col_name != "abcd"   |
| LIKE 		    |Case insensitive exact string comparison		     									|col_name LIKE "ABC"  |
| NOT LIKE 		    |Case insensitive exact string inequality comparison   								|col_name NOT LIKE "ABCD"|
| % 			    |Used anywhere in a string to match a sequence of zero or more characters (only with LIKE or NOT LIKE) 	|col_name LIKE "%AT%" <br /> (matches "AT", "ATTIC", "CAT" or even "BATS")|
| _ 			    |Used anywhere in a string to match a single character (only with LIKE or NOT LIKE) |col_name LIKE "AN_"		 <br /> (matches "AND", but not "AN")|
| IN (…) 		    |String exists in a list 														|col_name IN ("A", "B", "C")|
| NOT IN (…) 	    |String does not exist in a list 												|col_name NOT IN ("D", "E", "F)|

notes that full-text search is better left to deticated libraries like $\textbf{Apache Lucene}$ or $\textbf{Sphinx}$.

$\textbf{Ex}.$ to find all Toy Story movies:
``` sql
SELECT *
FROM movies
WHERE Title LIKE "%Toy Story%";
```

<h1 align="center">Filtering</h1> 

$\textbf{DISTINCT}:$ keyword that'll blindly discard duplicated values when querrying
``` sql
SELECT DISTINCT column1, column2, ...
FROM mytable
WHERE conditions;
```
Since the $\text{DISTINCT}$ keyword will blindly remove duplicate rows, we will learn in a future lesson how to discard duplicates based on specific columns using grouping and the $\text{GROUP BY}$ clause.

$\textbf{ORDER BY}:$ clause to sort results by a given column in ASCending or DESCending order.
``` sql
SELECT column1, column2, ...
FROM mytable
WHERE conditions
ORDER BY column1 ASC/DESC;
```
each row is sorted alpha-numerically based on the specified column's values, in some databases you can also specify a collation to better soty data containing multi-language text.

$\textbf{LIMIT and OFFSET}:$ clauses commonly used with $\text{ORDER BY}$. the $\text{LIMIT}$ will reduce the number of rows to return, and the optional $\text{OFFSET}$ will specify where to begin counting the number rows from.
``` sql
SELECT column1, column2, ...
FROM mytable
WHERE conditions
ORDER BY column1 ASC/DESC
LIMIT num_limit OFFSET num_offset;
```

$\textbf{Ex.}$ to list pixar movies sorted alphabetically offseted by 5:
``` sql
SELECT title FROM movies
ORDER BY title ASC
LIMIT 5 OFFSET 5;
```

<h1 align="center">Database Normalization</h1>  

going from single table -> multiple orthogonal tables, using normalization.

main purpose: removing duplicate entries while allowing the different tables to change independent of eachother. 

queries get more complex since they have to find data from different parts of the databases. there are thus also performence issues.


$\textbf{JOINs}:$  
tables that share info about a single entity need to have a $\it{primary \space key}$ that identifies that entity $\it{uniquely}$ across the database.

common primary key type is an auto-incrementing integer (space efficient), but it can be any unique key, such as string, hashed value, etc.

the $\text{JOIN}$ clause in a query can combine row data across two seperate tables using a unique key.

$\textbf{INNER JOIN}:$  
a process that matches rows from the first table and the second which have the same key (as defined by the $\text{ON}$ constraint). 

``` sql
SELECT column1, column2, ...
FROM mytable
INNER JOIN table2
    ON mytable.id = table2.id
WHERE conditions
ORDER BY column, ... ASC/DESC
LIMIT num_limit OFFSET num_offest;
```

this results in a row with combined columns from both tables. after the tables are joined, the other clauses are applied.

NOTE: $\text{INNER JOIN}$ can be written as $\text{JOIN}$, however we use the first to dinstinguish from other types of joins. 

$\textbf{Ex.}$ two tables, "Movies" and "Boxoffice" that are to be joined, want to show movies where int sales greater than domestic, ordered by rating desc:

``` sql
SELECT * FROM Movies
INNER JOIN Boxoffice
    ON Movies.Id = Boxoffice.Movie_id
WHERE International_sales > Domestic_sales
ORDER BY Rating DESC;
```

$\textbf{OUTER JOIN}:$  
if the two tables have asymmetric (?) data, then we have to use one of $\text{LEFT JOIN, RIGHT JOIN, FULL JOIN}.$ to ensure that needed data isn't discarded bc it only belongs to one of the tables.

when joining table A to table B:  
> $\textbf{LEFT JOIN}:$ keeps rows from $\text{A}$ regardless of whether a matching row is found in $\text{B}$.  
$\textbf{RIGHT JOIN}:$ keeps rows from $\text{B}$ regardless of whether a matching row is found in $\text{A}$.  
$\textbf{FULL JOIN}:$ keeps all rows from both tables.

when using any of those special joins, you'l likely need to write additional logic to deal with $\text{NULL}$'s in the result and constraints.

``` sql
SELECT column1, columns2, ...
FROM mytable
INNER/LEFT/RIGHT/FULL JOIN another_table
    ON mytable.id = another_table.matching
WHERE conditions
ORDER BY columns1, ... ASC/DESC
LIMIT num_limit OFFSET num_offset;
```

NOTE: sometimes there's $\text{OUTER}$ keyword, this is for older SQL compatibility.

$\textbf{Ex}.$ two tables, "Buildings" of bulding_name and capacity and "Employees" of role, name, building, years_employed. need to find list of all buildings that have employees.

``` sql
SELECT DISTINCT building_name FROM employees
LEFT JOIN buildings
    ON employees.building = buildings.building_name;
```

$\textbf{Ex}.$ same tables, need to list all buildings and the distinct employee roles in each building (including empty buildings) .

``` sql
SELECT DISTINCT building_name, role FROM buildings
LEFT JOIN Employees
    ON buildings.building_name = employees.building;
```

$\textbf{NULLs}:$  
a good alternative to $\text{NULL}$ values in a database is data-type appropriate default values, like 0 for numerical data, empty strings for text data, etc. but if you need to work with incomplete data, then $\text{NULL}$ values can be appropriate if teh default values will skew later analysis (ex. when taking averages of numerical data).

sometimes it's not possible to avoid $\text{NULL}$ values, as above when outer-joining two tables with asymmetric data. in those cases, you can test a column for $\text{NULL}$ values in a $\text{WHERE}$ clause by using either of $\text{IS NULL / IS NOT NULL}$ constraint.

select query with constraints on NULL values:
```sql
SELECT column1, column2, ...
FROM mytable
WHERE column1 IS/IS NOT NULL
AND/OR another_cond;
```

<h1 align="center">Queries with Expressions</h1>  

$\underline{\textbf{Expressions}}:$  
can use expressions to apply mathematical logic or string function on column values in a query.

ex. query with expressions
``` sql
SELECT particle_speed / 2.0 AS half_particle_speed
FROM physics_data
WHERE ABS(particle_position) * 10.0 > 500;
```

each database has its own supported set of mathematical, string, and date functions that can be used in a query, which you can find in their own respective docs.

the use of expressions can save time and post-processing, but can make the quert harder to read, when expressions are used in the $\text{SELECT}$ part of the query, they can be given an alias using the $\text{AS}$ keyword.

$\underline{\textbf{Aliases}}:$
``` sql
SELECT col_expression AS expr_alias, ...
FROM mytable;
```

regular columns and even tables can also have aliases:

``` sql
SELECT column AS col_alias, ...
FROM table_name AS table_alias
INNER JOIN table2
    ON table_alias.id = table2.id;
```

$\underline{\textbf{Aggregates (functions)}}:$

select query with aggregate functions over all rows
``` sql
SELECT AGG_FUNC(column_or_expr) AS aggregate_alias, ...
FROM mytable
WHERE constraint_expr
```

common aggregate funcitons:

| Function | Description
|----------|------------|
|COUNT(*)  |counts number of rows in the group if no column name is specificed|
|COUNT(column)| counts number of rows in the group with non-NULL values in the specified column|
|MIN(column)| finds smallest numerical value in specified column for all rows in the group|
|MAX(column)| finds largest numerical value in specified column for all rows in the group|
|AVG(column)| finds average numerical value in specified column for all rows in the group|
|SUM(column)| sums all numerical values in specified column for all rows in the group|

$\underline{\textbf{Grouped Aggregate Functions}}:$  
to apply aggregate funcitons to individual groups of data within that group (ex. box office sales for comedies vs actions), this would create as many results as there are unique groups defined as by the $\text{GROUP BY}$ clause.  

select query with aggregate functions over groups:
``` sql
SELECT AGG_FUNC(column_or_expr) AS aggregate_alias, ...
FROM mytable
WHERE constraint_expr
GROUP BY column;
```

$\textbf{GROUP BY}:$ groups rows that have the same values in the column specified.

$\textbf{Ex.}$ employee table with rolem name, building, year employeed. to find average number of years in each role:
``` sql
SELECT Name, Role, AVG(Years_employed) FROM employees
GROUP BY Role;
```

notice that if the $\text{GROUP BY}$ clause is executed after the $\text{WHERE}$ clause (which filters the rows which are to be grouped), then we need a way to filter the grouped rows. 

$\textbf{HAVING}:$  
we can add an additional $\text{HAVING}$ clause, which is used specifically with the $\text{GROUP BY}$ clause to allow us to filter grouped rows from the result set.

``` sql
SELECT group_by_col, AFF_FUNC(col_expr) AS aggregate_col
FROM mytable
WHERE conditions
GROUP BY column
HAVING group_cond
```

the $\text{HAVING}$ clause constraints are written the same way as with the $\text{WHERE}$ clause, and are applied to the grouped rows.

$\textbf{Ex.}$ employee table with columns of roles, names, buildings, years employed. to find total number of years employed by all engineers:

``` sql
SELECT Role, SUM(years_employed) AS total_num_years FROM
employees
GROUP BY Role
HAVING Role = "Engineer";
```

<h1 align="center">Order of Execution</h1>

we now have a template for a complete select query:
``` sql
SELECT DISTINCT column, AGG_FUNC(column_or_expression), …
FROM mytable
    JOIN another_table
      ON mytable.column = another_table.column
    WHERE constraint_expression
    GROUP BY column
    HAVING constraint_expression
    ORDER BY column ASC/DESC
    LIMIT count OFFSET COUNT;
```

each query begins with finding the data that we need in a database, and then filtering that data down into something that can be processed and understood as quickly as possible.

each part of the query is executed sequentially.

1. $\textbf{FROM}$ and $\textbf{JOINs}$
>the FROM clause, and subsequent JOINs are first executed to determine the total working set of data that is being queried. This includes subqueries in this clause, and can cause temporary tables to be created under the hood containing all the columns and rows of the tables being joined.

2. $\textbf{WHERE}$
>once we have the total working set of data, the first-pass WHERE constraints are applied to the individual rows, and rows that do not satisfy the constraint are discarded. Each of the constraints can only access columns directly from the tables requested in the FROM clause. Aliases in the SELECT part of the query are not accessible in most databases since they may include expressions dependent on parts of the query that have not yet executed.

3. $\textbf{GROUP BY}$
>the remaining rows after the WHERE constraints are applied are then grouped based on common values in the column specified in the GROUP BY clause. As a result of the grouping, there will only be as many rows as there are unique values in that column. Implicitly, this means that you should only need to use this when you have aggregate functions in your query.

4. $\textbf{HAVING}$
>if the query has a GROUP BY clause, then the constraints in the HAVING clause are then applied to the grouped rows, discard the grouped rows that don't satisfy the constraint. Like the WHERE clause, aliases are also not accessible from this step in most databases.

5. $\textbf{SELECT}$
>any expressions in the SELECT part of the query are finally computed.

6. $\textbf{DISTINCT}$
>of the remaining rows, rows with duplicate values in the column marked as DISTINCT will be discarded.

7. $\textbf{ORDER BY}$
>if an order is specified by the ORDER BY clause, the rows are then sorted by the specified data in either ascending or descending order. Since all the expressions in the SELECT part of the query have been computed, you can reference aliases in this clause.

8. $\textbf{LIMIT / OFFSET}$
>finally, the rows that fall outside the range specified by the LIMIT and OFFSET are discarded, leaving the final set of rows to be returned from the query


---
$\textbf{Example}:$  
available tables:

Table: Movies
| id | title               | director       | year | length_minutes |
|----|---------------------|----------------|------|----------------|
| 1  | Toy Story           | John Lasseter  | 1995 | 81             |
| 2  | A Bug's Life        | John Lasseter  | 1998 | 95             |
| 3  | Toy Story 2         | John Lasseter  | 1999 | 93             |
| 4  | Monsters, Inc.      | Pete Docter    | 2001 | 92             |
| 5  | Finding Nemo        | Andrew Stanton | 2003 | 107            |
| 6  | The Incredibles     | Brad Bird      | 2004 | 116            |
| 7  | Cars                | John Lasseter  | 2006 | 117            |
| 8  | Ratatouille         | Brad Bird      | 2007 | 115            |
| 9  | WALL-E              | Andrew Stanton | 2008 | 104            |
| 10 | Up                  | Pete Docter    | 2009 | 101            |
| 11 | Toy Story 3         | Lee Unkrich    | 2010 | 103            |
| 12 | Cars 2              | John Lasseter  | 2011 | 120            |
| 13 | Brave               | Brenda Chapman | 2012 | 102            |
| 14 | Monsters University | Dan Scanlon    | 2013 | 110            |


Table: Boxoffice

| movie_id | rating | domestic_sales | international_sales |
|----------|--------|----------------|---------------------|
| 5        | 8.2    | 380843261      | 555900000           |
| 14       | 7.4    | 268492764      | 475066843           |
| 8        | 8      | 206445654      | 417277164           |
| 12       | 6.4    | 191452396      | 368400000           |
| 3        | 7.9    | 245852179      | 239163000           |
| 6        | 8      | 261441092      | 370001000           |
| 9        | 8.5    | 223808164      | 297503696           |
| 11       | 8.4    | 415004880      | 648167031           |
| 1        | 8.3    | 191796233      | 170162503           |
| 7        | 7.2    | 244082982      | 217900167           |
| 10       | 8.3    | 293004164      | 438338580           |
| 4        | 8.1    | 289916256      | 272900000           |
| 2        | 7.2    | 162798565      | 200600000           |
| 13       | 7.2    | 237283207      | 301700000           |

to find the total domestic and international sales attributed to each direction:

``` sql
SELECT Director, SUM(International_sales+Domestic_sales)
AS total_sales
FROM movies
LEFT JOIN Boxoffice
    ON Movies.Id = Boxoffice.Movie_id
GROUP BY Director;
```

resulting in:
|Director       |Total_sales |
|---------------|------------|
|Andrew Stanton	|1458055121  |
|Brad Bird      |1255164910  |
|Brenda Chapman	|538983207   |
|Dan Scanlon	|743559607   |
|John Lasseter	|2232208025  |
|Lee Unkrich	|1063171911  |
|Pete Docter	|1294159000  | 

---

<h1 align="center">Inserting Rows</h1>

$\underline{\textbf{Database Schema}}:$  
what described the structure of each table, and the datatypes each column of the table can contain.